In [119]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler
import pysal as ps 
from pysal.model.spreg import OLS 
from pysal.lib.weights import KNN
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression 
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split
import numpy as np

In [92]:
comparable_geocode = pd.read_csv('comparablerental_geocode.csv')
comparable_rentals = pd.read_csv('comparablerentals.csv')

In [93]:
comparable_complete = [comparable_rentals, comparable_geocode]
comparable_complete = pd.concat(comparable_complete, axis=1)
no_na_comparable_complete = comparable_complete.dropna()

In [94]:
no_na_comparable_complete.columns

Index(['Boro-Block-Lot', 'Address', 'Neighborhood', 'Building Classification',
       'Total Units', 'Year Built', 'Gross SqFt', 'Estimated Gross Income',
       'Gross Income per SqFt', 'Estimated Expense', 'Expense per SqFt',
       'Net Operating Income', 'Full Market Value', 'Market Value per SqFt',
       'Distance from Co-op in miles', 'Report Year', 'Estimated',
       'Condo Reference', 'Borough', 'Borough PCA 1', 'Borough PCA 2',
       'Borough PCA 3', 'Address', 'Latitude', 'Longitude',
       'Standard Address'],
      dtype='object')

In [95]:
geo_coords = no_na_comparable_complete[['Longitude', 'Latitude']]
ys = pd.DataFrame(no_na_comparable_complete[['Gross Income per SqFt', 'Estimated Gross Income']])
no_na_comparable_complete.drop(columns=['Boro-Block-Lot', 'Address', 'Neighborhood', 'Building Classification', 'Estimated Gross Income', 'Gross Income per SqFt', 'Estimated Expense', 'Expense per SqFt', 'Net Operating Income', 'Distance from Co-op in miles', 'Estimated', 'Condo Reference', 'Borough', 'Address', 'Standard Address', 'Latitude', 'Longitude'], axis=1, inplace=True)


/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [96]:
x = no_na_comparable_complete
ss = StandardScaler() 
 
x = ss.fit_transform(x)
x = pd.DataFrame(x, columns=['Total Units', 'Year Built', 'Gross SqFt', 'Full Market Value', 'Market Value per SqFt',\
                            'Report Year', 'Borough PCA 1', 'Borough PCA 2', 'Borough PCA 3'])
ys = ss.fit_transform(ys)
ys= pd.DataFrame(ys, columns=['Gross Income per SqFt', 'Estimated Gross Income'])

y1 = ys['Gross Income per SqFt']
y2 = ys['Estimated Gross Income']


In [86]:
sqft_x_train, sqft_x_test, sqft_y_train, sqft_y_test = train_test_split(x, y1, train_size=0.3, random_state=3)
total_x_train, total_x_test, total_y_train, total_y_test = train_test_split(x, y2, train_size=0.3, random_state=3)

In [87]:
mutual_reg = SelectKBest(mutual_info_regression, k=6)
mutual_reg.fit(total_x_train, total_y_train.ravel())

f_reg = SelectKBest(f_regression, k=6)
f_reg.fit(total_x_train, total_y_train.ravel())


SelectKBest(k=6, score_func=<function f_regression at 0x1a2be57510>)

In [88]:
#total income 
kbest = pd.DataFrame([x.columns, mutual_reg.scores_, f_reg.scores_], 
                     index=['feature', 'mutual_reg', 'f _reg']).T.sort_values('mutual_reg', ascending=False)
kbest

,feature,mutual_reg,f _reg
3,Full Market Value,1.73651,205957
2,Gross SqFt,1.57055,76492
0,Total Units,1.20091,36513.6
4,Market Value per SqFt,0.796539,2173.11
1,Year Built,0.634415,3681.72
8,Borough PCA 3,0.180753,66.8324
7,Borough PCA 2,0.180017,148.851
6,Borough PCA 1,0.179978,1548.8
5,Report Year,0.178546,66.2048


In [89]:
sqft_mutual_reg = SelectKBest(mutual_info_regression, k=5)
sqft_mutual_reg.fit(sqft_x_train, sqft_y_train.ravel())

sqft_f_reg = SelectKBest(f_regression, k=5)
sqft_f_reg.fit(sqft_x_train, sqft_y_train.ravel())

SelectKBest(k=5, score_func=<function f_regression at 0x1a2be57510>)

In [90]:
#income per sqft 
kbest = pd.DataFrame([x.columns, sqft_mutual_reg.scores_, sqft_f_reg.scores_], 
                     index=['feature', 'mutual_reg', 'f_reg']).T.sort_values('mutual_reg', ascending=False)
kbest

,feature,mutual_reg,f_reg
4,Market Value per SqFt,1.30123,77695
3,Full Market Value,0.692901,4128
2,Gross SqFt,0.599117,8.88808
8,Borough PCA 3,0.478904,788.634
7,Borough PCA 2,0.478154,59.7956
6,Borough PCA 1,0.477391,25929.4
0,Total Units,0.426595,10.6208
1,Year Built,0.410134,502.433
5,Report Year,0.211095,1397.85


In [133]:
geo_coords = geo_coords.rename(columns={'Longitude':'x', 'Latitude':'y'})

In [150]:
geo_coords.shape

(85374, 2)

In [270]:
len(geo_coords['x'].iloc[0]) 
#geo_coords['y'].iloc[0]

TypeError: object of type 'numpy.float64' has no len()

In [314]:
data = list(zip(geo_coords['x'], geo_coords['y']))

In [315]:
data = data[1000:4000]

IndexError: tuple index out of range

In [243]:
#kd = ps.lib.cg.kdtree.KDTree(np.array(data))

In [317]:
wnn3 = ps.lib.weights.KNN(data, k=3)

ValueError: list.remove(x): x not in list

IndexError: list index out of range

In [245]:
data2 = data[0::100]

In [246]:
wnn4 = ps.lib.weights.KNN(data2, k=3)

IndexError: list index out of range

In [153]:
points = [(10, 10), (20, 10), (40, 10), (15, 20), (30, 20), (30, 30)]
kd = ps.lib.cg.kdtree.KDTree(np.array(points))
wnn2 = ps.lib.weights.KNN(kd, 2)

In [156]:
wnn2[0]

{1: 1.0, 3: 1.0}